# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [2]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("../../data/proofs/english/problem3/proof1.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()


# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [3]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Triplet(entities=[Entity(id='theorem_3.10', name='Theorem 3.10', label='Theorem', type='Mathematical Statement'), Entity(id='matrix_A', name='Matrix A', label='Matrix', type='Matrix'), Entity(id='matrix_A_inverse', name='Matrix A^{-1}', label='Inverse Matrix', type='Matrix'), Entity(id='matrix_B', name='Matrix B', label='Matrix', type='Matrix'), Entity(id='identity_matrix', name='Identity Matrix I', label='Matrix', type='Matrix'), Entity(id='solution', name='A^{-1} B', label='Solution', type='Expression'), Entity(id='solution_v', name='v', label='Solution', type='Variable')], relations=[Relation(source='theorem_3.10', target='matrix_A', type='conditions', name='A is invertible'), Relation(source='matrix_A', target='solution', type='results_in', name='gives unique solution'), Relation(source='matrix_A_inverse', target='solution', type='results_in', name='provides solution'), Relation(source='solution_v', target='matrix_B', type='equivalence', name='is a solution'), Relation(source='solu

## 3. Store Triplets into Neo4J DB

In [ ]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("fanavaran", "fanavaran"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()